In [2]:
import sqlite3
import numpy as np
conn = sqlite3.connect('boston.db')
c = conn.cursor()

In [14]:
all_data = c.execute('SELECT * from boston').fetchall()

In [15]:
print type(all_data)
print len(all_data)
print type(all_data[0])
print len(all_data[0])

<type 'list'>
506
<type 'tuple'>
14


In [17]:
Xy = np.array(all_data)
X = Xy[:,:-1]
y = Xy[:,-1]

In [18]:
print X.shape
print y.shape

(506, 13)
(506,)


In [3]:
conn.rollback() #revert database to last commit
c = conn.cursor()

In [23]:
print c.execute('SELECT * FROM sqlite_master').fetchall()

[(u'table', u'boston', u'boston', 2, u'CREATE TABLE boston (CRIM REAL,ZN REAL,INDUS REAL,CHAS REAL,NOX REAL,RM REAL,AGE REAL,DIS REAL,RAD REAL,TAX REAL,PTRATIO REAL,B REAL,LSTAT REAL,MEDV REAL)')]


In [4]:
everything = c.execute('SELECT * from boston')

In [5]:
#description returns column names for all columns in last query
names = [ i[0] for i in everything.description]
print len(names)
print names

14
['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']


In [6]:
medv = c.execute('SELECT MEDV FROM boston').fetchall()

In [7]:
medv = np.array(medv)
print medv.shape

(506, 1)


In [8]:
mean_medv = np.sum(medv)/float(len(medv))
var_medv = np.dot(np.transpose(medv-mean_medv),medv-mean_medv)[0,0]/float(len(medv)-1)
print 'mean of medv ',mean_medv
print 'var of medv ',var_medv

mean of medv  22.5328063241
var of medv  84.5867235941


In [9]:
tax_low = c.execute('SELECT TAX FROM boston WHERE MEDV<'+str(mean_medv)).fetchall()

In [10]:
tax_high = c.execute('SELECT TAX FROM boston WHERE MEDV>'+str(mean_medv)).fetchall()

In [11]:
tax_low = np.array(tax_low)
mean_tax_low = np.sum(tax_low)/float(len(tax_low))

In [12]:
tax_high = np.array(tax_high)
mean_tax_high = np.sum(tax_high)/float(len(tax_high))

In [13]:
print 'mean tax for lower than avg medv ',mean_tax_low
print 'mean tax for higher than avg medv ',mean_tax_high

mean tax for lower than avg medv  465.181818182
mean tax for higher than avg medv  327.315789474


In [14]:
unique_chas = c.execute('SELECT DISTINCT CHAS FROM boston').fetchall()

In [15]:
print unique_chas

[(0.0,), (1.0,)]


In [16]:
tax_high_1chas = c.execute('SELECT TAX FROM boston WHERE MEDV>'+str(mean_medv)+' AND CHAS = 1.0').fetchall()

In [17]:
tax_high_0chas = c.execute('SELECT TAX FROM boston WHERE MEDV>'+str(mean_medv)+' AND CHAS = 0.0').fetchall()

In [18]:
tax_high_1chas = np.array(tax_high_1chas)
mean_tax_high_1chas = np.sum(tax_high_1chas)/float(len(tax_high_1chas))

In [19]:
tax_high_0chas = np.array(tax_high_0chas)
mean_tax_high_0chas = np.sum(tax_high_0chas)/float(len(tax_high_0chas))

In [20]:
print (len(tax_high),len(tax_high_1chas),len(tax_high_0chas))
print (mean_tax_high,mean_tax_high_1chas,mean_tax_high_0chas)

(209, 20, 189)
(327.31578947368422, 366.44999999999999, 323.17460317460319)


In [82]:
highest_value_entries = c.execute('SELECT TAX,MEDV FROM boston ORDER BY MEDV DESC, TAX DESC').fetchmany(20)

In [83]:
highest_value_entries = np.array(highest_value_entries)

In [84]:
print highest_value_entries

[[ 777.7   55. ]
 [ 777.7   55. ]
 [ 666.    50. ]
 [ 666.    50. ]
 [ 666.    50. ]
 [ 666.    50. ]
 [ 666.    50. ]
 [ 403.    50. ]
 [ 403.    50. ]
 [ 403.    50. ]
 [ 403.    50. ]
 [ 307.    50. ]
 [ 264.    50. ]
 [ 264.    50. ]
 [ 255.    50. ]
 [ 224.    50. ]
 [ 264.    48.8]
 [ 224.    48.5]
 [ 307.    48.3]
 [ 307.    46.7]]


In [77]:
c.execute('INSERT INTO boston (TAX,MEDV) VALUES (999.0,77.0)')
c.execute('UPDATE boston SET TAX = 777.7,MEDV = 55.0 WHERE (MEDV=50.0 AND TAX<200.0)')

In [81]:
c.execute('DELETE FROM boston WHERE TAX = 999.0')

In [87]:
conn.rollback() #revert database to last commit
c = conn.cursor()